In [1]:
import kagglehub

# Download latest version
path = kagglehub.model_download("metaresearch/llama-3/transformers/8b-chat-hf")

print("Path to model files:", path)

Attaching model 'metaresearch/llama-3/transformers/8b-chat-hf' to your Kaggle notebook...


Path to model files: /kaggle/input/llama-3/transformers/8b-chat-hf/1


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lmsys-chatbot-arena/sample_submission.csv
/kaggle/input/lmsys-chatbot-arena/train.csv
/kaggle/input/lmsys-chatbot-arena/test.csv
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.safetensors.index.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00003-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf/1/config.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/LICENSE
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00001-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/USE_POLICY.md
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer_config.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/example_text_completion.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/test_tokenizer.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/requirements.txt
/kaggle/input/llama-3/transformers/8b-chat-h

In [3]:
!pip install accelerate bitsandbytes transformers peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 16.8 MB/s eta 0:00:00


In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import Dataset

import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig, Trainer, TrainingArguments
from peft import get_peft_model, PeftModel, PeftConfig, LoraConfig, TaskType, prepare_model_for_kbit_training

from torch.cuda.amp import autocast
from threading import Thread

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, log_loss

2024-06-04 08:16:08.056982: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-04 08:16:08.057095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-04 08:16:08.159909: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

#  Fix random seed

In [6]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Load data

In [7]:
df_train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
df_test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
print(df_train.shape)
print(df_test.shape)

# df_train = df_train[:100]

(57477, 9)
(3, 4)


# tokenizer

In [8]:
base_model_path = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'

tokenizer = AutoTokenizer.from_pretrained(base_model_path)
tokenizer.pad_token_id = 0
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# pytorch dataset

In [9]:
# Define MAX_LENGTH
MAX_LENGTH = 128

# Tokenize the text data
def tokenize_texts(texts, tokenizer, max_length=MAX_LENGTH):
    tokens = tokenizer(texts.tolist(), padding='max_length', max_length=max_length, truncation=True, return_tensors='pt')
    return tokens['input_ids'], tokens['attention_mask']

In [10]:
# create pytorch dataset for transformers Trainer

from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels=None):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        item = {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx]
        }
        if self.labels is not None:
            item['labels'] = self.labels[idx]
        return item

In [11]:
df_train = df_train[:100]

In [12]:
columns_to_vectorize = ['prompt', 'response_a', 'response_b']
df_train['text'] = df_train[columns_to_vectorize].astype(str).apply(lambda x: ' '.join(x), axis=1)
df_test['text'] = df_test[columns_to_vectorize].astype(str).apply(lambda x: ' '.join(x), axis=1)


target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']
y = df_train[target_columns].idxmax(axis=1)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(df_train['text'], y_encoded, test_size=0.1, random_state=42)
print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)

# Tokenize the texts
input_ids_train, attention_masks_train = tokenize_texts(x_train, tokenizer)
input_ids_val, attention_masks_val = tokenize_texts(x_val, tokenizer)
input_ids_test, attention_masks_test = tokenize_texts(df_test['text'], tokenizer)

(90,) (10,) (90,) (10,)


In [13]:
# from torch.utils.data import DataLoader

# Create the datasets
train_dataset = TextDataset(input_ids_train, attention_masks_train, torch.tensor(y_train))
val_dataset = TextDataset(input_ids_val, attention_masks_val, torch.tensor(y_val))
test_dataset = TextDataset(input_ids_test, attention_masks_test)

## Create the dataloaders
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
# test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# PEFT

In [14]:

from_ckpt = False  
output_dir = "lmsys"
ckpt_dir = os.path.join(output_dir, "checkpoints")
ckpt_name = "llama3"

base_model_path = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
cache_dir = "./cache"

os.makedirs(output_dir, exist_ok=True)
os.makedirs(ckpt_dir, exist_ok=True)


nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

peft_config = LoraConfig(
    r=2,
    lora_alpha=16,
    target_modules=['o_proj'],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_path,
    num_labels=len(target_columns),
    quantization_config=nf4_config,
    low_cpu_mem_usage = True
)
base_model.config.pad_token_id = tokenizer.pad_token_id

if from_ckpt:
    model = PeftModel.from_pretrained(base_model, ckpt_name)
else:
    model = base_model

# model = prepare_model_for_int8_training(model)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/llama-3/transformers/8b-chat-hf/1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model.print_trainable_parameters()

trainable params: 536,576 || all params: 7,505,473,536 || trainable%: 0.0071


In [16]:
training_args = TrainingArguments(
    output_dir=ckpt_dir,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    save_strategy="steps",
    save_steps=1,
    save_total_limit=5,
    metric_for_best_model='accuracy',
    report_to = 'none',
    fp16=True,  # Enable mixed precision training
    gradient_checkpointing=True  # Enable gradient checkpointing
)

In [17]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

In [18]:
import gc
gc.collect()

51

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,    
    compute_metrics=compute_metrics,
)

model.config.use_cache = False

# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/llama-3/transformers/8b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/llama-3/transformers/8b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: User

TrainOutput(global_step=15, training_loss=2.8292452494303384, metrics={'train_runtime': 248.6765, 'train_samples_per_second': 1.086, 'train_steps_per_second': 0.06, 'total_flos': 1265135863726080.0, 'train_loss': 2.8292452494303384, 'epoch': 2.608695652173913})

# testing data

In [20]:
# Make predictions on the test set
predictions = trainer.predict(test_dataset)
preds_test = predictions.predictions

# Since preds_test contains logits, apply softmax to get probabilities
import torch.nn.functional as F
preds_test_probabilities = F.softmax(torch.tensor(preds_test), dim=-1).numpy()

# Create a submission DataFrame
submission = pd.DataFrame({
    'id': df_test["id"],
    'winner_model_a': preds_test_probabilities[:, 0],
    'winner_model_b': preds_test_probabilities[:, 1], 
    'winner_tie': preds_test_probabilities[:, 2]
})

# Save the submission DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

# Print the first few rows of the submission DataFrame
print(submission.head())

        id  winner_model_a  winner_model_b  winner_tie
0   136060        0.010437        0.723144    0.266420
1   211333        0.007062        0.017718    0.975220
2  1233961        0.878157        0.000179    0.121664


In [21]:
tokenizer.save_pretrained(os.path.join(output_dir, "tokenizer"))

('lmsys/tokenizer/tokenizer_config.json',
 'lmsys/tokenizer/special_tokens_map.json',
 'lmsys/tokenizer/tokenizer.json')

In [22]:
model.save_pretrained(os.path.join(output_dir, "model"))

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/llama-3/transformers/8b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(
